# Initialization

Add the parent directory to the path, to emulate running the notebook from the top-level folder. This helps to keep imports understandable.

In [ ]:
import sys
sys.path.append('..')

Import the code which help structure the notebook and allow for easier data analysis, along with some default values to use for functions.

In [ ]:
from core.utilities.notebook_setups.frame_tools import *
from core.utilities import logging
from core.data_graphing import *

SET = 'MKM'
ROLL = 3
DERIVATIONS = 0
SRT_TRG = 'ALSA Change'
SAVE = False

Load the data for the set (or automatically fetch it if needed).

In [ ]:
data_manager, set_data = set_up(log_lvl=LogLvl.SPARSE, target_set=SET)
examiner = DataFrameFuncs(set_data.BO1)
frame = examiner.card_frame(deck_color='', summary=True)

# Data Overview

## Top Performing Cards

### All Cards

In [ ]:
examiner.get_top_performers(None, 25, 10, '')

### Colour-Pairs

In [ ]:
examiner.get_top_performers(['WU', 'WB', 'WR', 'WG', 'UB', 'UR', 'UG', 'BR', 'BG', 'RG'])

### By Colour

In [ ]:
examiner.get_top_performers('W')

In [ ]:
examiner.get_top_performers('U')

In [ ]:
examiner.get_top_performers('B')

In [ ]:
examiner.get_top_performers('R')

In [ ]:
examiner.get_top_performers('G')

## Archetype Playrate & Winrate History

### Overview

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.TWO_COLOR, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.TWO_COLOR, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.ALLIED, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.ALLIED, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.ENEMY, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.ENEMY, roll=ROLL)

### By Colour

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.WHITE, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.WHITE, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.BLUE, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.BLUE, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.BLACK, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.BLACK, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.RED, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.RED, roll=ROLL)

In [ ]:
examiner.plot_archetype_winrate_history(DefaultPlotConfigs.GREEN, roll=ROLL)
examiner.plot_archetype_playrate_history(DefaultPlotConfigs.GREEN, roll=ROLL)

## Card Pick Order Changes

In [ ]:
analyzer = PickOrderAnalyzer(examiner.DATA, SRT_TRG)
styled_frames = analyzer.gen_card_evaluation_summary()

### Commons

In [ ]:
styled_frames["ContestedCommons"]

In [ ]:
styled_frames["UncontestedCommons"]

### Uncommons

In [ ]:
styled_frames["ContestedUncommons"]

In [ ]:
styled_frames["UncontestedUncommons"]

### Top Performers

In [ ]:
styled_frames["TopPerformingCommons"]

In [ ]:
styled_frames["TopPerformingUncommons"]

### Key Wheelable Cards

In [ ]:
wheelable_cards = analyzer.get_wheelable_summary(save=SAVE)
wheelable_cards['']

In [ ]:
raise Exception('Stopping Auto-Run!')

# Color Performance Analysis

## ???

Need to rework this section to provide a clear overview of what makes each colour (or colour-pair) tick.

In [ ]:
analyzer = PickOrderAnalyzer(examiner.DATA, SRT_TRG)
wheelable_cards = analyzer.get_wheelable_summary(save=SAVE)
wheelable_iterator = iter(wheelable_cards.values())

In [ ]:
next(wheelable_iterator)

In [ ]:
examiner.get_top_performers('', deck_color='GW')

In [ ]:
plot_config = DefaultPlotConfigs.RED
examiner.plot_archetype_winrate_history(plot_config, roll=ROLL, derivs=DERIVATIONS)
examiner.plot_archetype_playrate_history(plot_config, roll=ROLL, derivs=DERIVATIONS)

In [ ]:
raise Exception('Stopping Auto-Run!')

# Examine Data

NOTE: Unsure if these should remain here, or be moved to some more specific notebook.

In [ ]:
prettify_frame(examiner.stat_archetype_performance('GIH WR'))

In [ ]:
prettify_frame(examiner.card_relative_winrates('WR', 'GIH WR'))

In [ ]:
prettify_frame(examiner.card_archetype_performance("Novice Inspector"))

In [ ]:
raise Exception('Stopping Auto-Run!')